In [1]:
import Pkg;
Pkg.add("PyPlot");
Pkg.add("LinearAlgebra");
Pkg.add("OSQP");
Pkg.add("BlockDiagonals");

    Updating registry at `C:\Users\Visagio\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Manifest.toml`


In [2]:
# b = [1,1] - a vírgula faz ser um vetor coluna
waypts = [[0,0],
        [1,2],
        [2,-1],
        [4,8],
        [5,2]];

v0 = [0,0];
a0 = [0,0];
v1 = [0,0];
a1 = [0,0];
T = 5;
n_order = 5;

In [3]:
function computeQ(n, r, t1, t2)
    T = [0.0 for i in range(1, (n-r)*2+1 )];
    
    for i in range(1,(n-r)*2+1)
        T[i] =  round(t2^i - t1^i, digits = 4) # 4 dígitos é a quantidade usada no matlab
    end
    
    # Q = [[0.0 for i in range(1, n+1)] for i in range(1, n+1)] # equivalente ao zeros(n+1) do matlab / produz uma matriz de zeros
    Q = zeros(n+1, n+1)
    
        for i in range(r+1, n+1)
            for j in range(i, n+1)
                k1 = i-r-1
                k2 = j-r-1
                k = k1+k2+1
                Q[i,j] = prod(k1+1:k1+r)*prod(k2+1:k2+r)/k*T[k];
                # Q[j][i] = Q[i][j];
            end
        end
        
        return Q
    end
    
n = 5; r = 3; t1 = 0.0 ; t2 = 0.5401 # argumentos de teste
Q = computeQ(n, r, t1, t2)
Q

6×6 Matrix{Float64}:
 0.0  0.0  0.0   0.0      0.0      0.0
 0.0  0.0  0.0   0.0      0.0      0.0
 0.0  0.0  0.0   0.0      0.0      0.0
 0.0  0.0  0.0  19.4436  21.0024  18.912
 0.0  0.0  0.0   0.0     30.2592  30.636
 0.0  0.0  0.0   0.0      0.0     33.12

In [4]:
# Calculando o Q_all : função de custo
n_poly = 3
ts = [1, 2, 3, 4, 5]
Q_all = sparse(computeQ(n_order,3,ts[1],ts[1+1]) );

for i = 2:n_poly
    Q_all = BlockDiagonal([sparse(Q_all), sparse(computeQ(n_order,3,ts[i],ts[i+1]) )] );
end
Q_all

UndefVarError: UndefVarError: sparse not defined

In [5]:
b_all = zeros(size(Q_all, 1), 1);

UndefVarError: UndefVarError: Q_all not defined

In [6]:
function calc_tvec(t,n_order,r)
    
    tvec = zeros(1,n_order+1);
    for i=range(r+1,n_order+1)
        tvec[i] = prod(range(i-r,i-1))*t^(i-r-1);
    end

    return tvec
end
t = 1; n_order = 4; r = 3;
tvec = calc_tvec(t, n_order, r)
tvec

1×5 Matrix{Float64}:
 0.0  0.0  0.0  6.0  24.0

In [7]:
# Construção da matriz de restrições
# Ainda incompleto

n_order = 5
n_coef = n_order + 1
n_poly = length(waypts) - 1

ts[1] = 0
ts[end] = 5
Aeq = zeros(4*n_poly+2,n_coef*n_poly);

# Aeq
Aeq[range(1,3),range(1,n_coef)] = [
    calc_tvec(ts[1],n_order,0);
    calc_tvec(ts[1],n_order,1);
    calc_tvec(ts[1],n_order,2)];
    
    Aeq[range(4,6),range(n_coef*(n_poly-1)+1,n_coef*n_poly)] = [
        calc_tvec(ts[end],n_order,0);
        calc_tvec(ts[end],n_order,1);
        calc_tvec(ts[end],n_order,2)];
        
Aeq

18×24 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0   0.0    0.0    0.0     0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0.0     0.0
 0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0.0     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     5.0  25.0  125.0  625.0  3125.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  10.0   75.0  500.0  3125.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0   2.0   30.0  300.0  2500.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0.0     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0.0     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0.0     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0.0     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0   0.0    0.0    0.0     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0.0     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0   0.0    0.0    0